In [1]:
import numpy as np
import plotly.graph_objects as go

# === Paramètres ===
Te = 1.0
T = 100  # nombre de pas de temps

# Matrice de transition (état : x, y, vx, vy)
F = np.array([
    [1, 0, Te,  0],
    [0, 1,  0, Te],
    [0, 0,  1,  0],
    [0, 0,  0,  1]
])

# Matrice d'observation (on observe x, y)
H = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

# Covariances
Q =  np.eye(4)  # bruit de processus
R = 30 * np.eye(2)  # bruit de mesure

# État initial
x_true = np.array([0, 0, 1, 1])
x_est = np.array([0, 0, 0, 0])
P = 100 * np.eye(4)  # forte incertitude initiale

kl_hist = []

# === Fonction de divergence KL entre 2 lois normales multivariées ===
def kl_divergence_gaussians(mu0, P0, mu1, P1):
    n = mu0.shape[0]
    inv_P1 = np.linalg.inv(P1)
    delta = mu1 - mu0
    term1 = np.trace(inv_P1 @ P0)
    term2 = delta.T @ inv_P1 @ delta
    sign0, logdet0 = np.linalg.slogdet(P0)
    sign1, logdet1 = np.linalg.slogdet(P1)
    if sign0 <= 0 or sign1 <= 0:
        return np.nan
    term3 = logdet1 - logdet0
    return 0.5 * (term1 + term2 - n + term3)

# === Boucle de filtrage Kalman ===
for _ in range(T):
    # Mouvement réel
    w = np.random.multivariate_normal(np.zeros(4), Q)
    x_true = F @ x_true + w

    # Observation
    v = np.random.multivariate_normal(np.zeros(2), R)
    z = H @ x_true + v

    # Prédiction
    x_pred = F @ x_est
    P_pred = F @ P @ F.T + Q

    # Mise à jour
    y = z - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    # KL divergence entre postérieur et prédiction
    kl = kl_divergence_gaussians(x_est, P, x_pred, P_pred)
    kl_hist.append(kl)

# === Affichage Plotly ===
fig_kl = go.Figure()
fig_kl.add_trace(go.Scatter(
    x=list(range(T)),
    y=kl_hist,
    mode='lines',
    name='KL Divergence'
))
fig_kl.update_layout(
    title="Divergence de Kullback-Leibler entre prédiction et mise à jour (",
    xaxis_title="Temps",
    yaxis_title="D_KL (nats)",
    template="plotly_white"
)
fig_kl.show()
